In [1]:
from har_datasets.parsing.downloading import download

In [2]:
download(
    url="https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip",
    save_dir="./",
)

File human+activity+recognition+using+smartphones.zip already exists in ./
Directory ./human+activity+recognition+using+smartphones already exists and is not empty
